In [1]:
import sagemaker
import boto3
import json

boto_session = boto3.Session(profile_name='dev-profile')
# sagemaker_session = sagemaker.Session(boto_session=session)

# iam = boto_session.resource('iam')
# role = iam.get_role(RoleName='sagemaker-execution-role')['Role']['Arn']

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/katie/Library/Application Support/sagemaker/config.yaml


Try: aws sagemaker-runtime invoke-endpoint --endpoint-name stablelm-zephyr-3b-endpoint --body '{"inputs": "What is the capital of France?"}' --content-type application/json output.json


from transformers import DistilBertTokenizer, TFDistilBertForQuestionAnswering     
import tensorflow as tf     

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-distilled-squad")     
model = TFDistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad")     

question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"     

inputs = tokenizer(question, text, return_tensors="tf")      
outputs = model(**inputs)     

answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])     
answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])     

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]     
tokenizer.decode(predict_answer_tokens)     

In [4]:
client = boto_session.client("sagemaker-runtime")
 
# You can also get the endpoint name by inspecting the output of Terraform with terraform output 
# or going to the SageMaker service in the AWS Management console. 
ENDPOINT_NAME = "distilbert-model-endpoint"
 
body={"question":"List 3 synonyms for the word tiny.", "context":"Synonyms for tiny are small, minute, small-scale, mini, baby, fun-size, petite."}
response = client.invoke_endpoint(
            EndpointName=ENDPOINT_NAME,
            ContentType="application/json",
            Accept="application/json",
            Body=json.dumps(body),
        )
print(response['Body'].read().decode('utf-8'))

{"score":0.506294310092926,"start":0,"end":26,"answer":"you are a smart dictionary"}
